# [역설계 및 복원 평가] 분류 및 연속모델 복합 구조 cVAE 촉매 최적화
### NVAE(2020) 기반 KL Balancing [0.01, 0.005, 0.002] 및 Recon 1:1:1 가중치 반영

In [ ]:
import os, sys, torch, json
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# --- [중요] 경로 설정 및 커스텀 모듈 임포트 ---
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from multilayer_model.m3_multi_bce import M3_Multi_BCE
from multilayer_model.m3_multi_mse import M3_Multi_MSE
from multilayer_loss.l_multi3_final_logic import l_multi3_final_loss
from vae_earlystopping import EarlyStopping

# --- 0. 데이터 로드 (에러 방지를 위해 루프 밖에서 선언) ---
try:
    x1_raw = np.load('../data/metal.npy')
    x2_raw = np.load('../data/support_norm.npy')
    x3_raw = np.load('../data/pre_fin.npy')
    c_raw = np.load('../data/re_fin.npy')
    print("✅ 데이터 로드 성공!")
except FileNotFoundError as e:
    print(f"❌ 데이터를 찾을 수 없습니다. 경로를 확인하세요: {e}")
    sys.exit()

# --- 1. 유틸리티 함수 정의 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_RUNS = 20
EPOCHS = 800
all_run_r2 = []

def to_t(a): return torch.tensor(a, dtype=torch.float32)
def prep(d, i, s, fit=False): 
    return s.fit_transform(d[i]) if fit else s.transform(d[i])

def train_expert(model, mode, train_loader, val_loader):
    opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    es = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            opt.zero_grad()
            p, m, v = model(b1, b2, b3, bc)
            # KL Weight: [0.01, 0.005, 0.002] 적용
            loss = l_multi3_final_loss(p, [b1, b2, b3], m, v, mode=mode, gamma_list=[0.01, 0.005, 0.002])
            loss.backward(); opt.step()
        
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vp, vm, vv = model(v1, v2, v3, vc)
                v_loss += l_multi3_final_loss(vp, [v1, v2, v3], vm, vv, mode=mode).item()
        
        # EarlyStopping 체크
        if es(v_loss/len(val_loader), model): break
    
    es.load_best_model(model)
    return model

# --- 2. Main 반복 루프 (20 Runs) ---
print(f"🚀 Starting {NUM_RUNS} Independent Runs on {device}...")

for run in range(1, NUM_RUNS + 1):
    # 매 실행마다 다른 시드로 데이터 분할 (변동성 확보)
    seed = 42 + run
    idx = np.arange(len(x1_raw))
    tr_idx, te_idx = train_test_split(idx, test_size=0.1, random_state=seed)
    tr_idx, va_idx = train_test_split(tr_idx, test_size=0.1, random_state=seed)

    # 스케일링 (Data Leakage 방지를 위해 매번 초기화)
    sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
    x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
    x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
    x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
    c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

    train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
    test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

    # 모델 차원 및 생성
    x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]
    c_dim = c_tr.shape[1]

    # 모델 학습
    m_bce = train_expert(M3_Multi_BCE(x_dims, c_dim).to(device), 'bce', train_loader, val_loader)
    m_mse = train_expert(M3_Multi_MSE(x_dims, c_dim).to(device), 'mse', train_loader, val_loader)

    # 평가 루틴
    m_bce.eval(); m_mse.eval()
    all_re_gen, all_re_true = [], []

    with torch.no_grad():
        for b1, b2, b3, bc in test_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            b_re_logits, _, _ = m_bce(b1, b2, b3, bc)
            m_re_vals, _, _ = m_mse(b1, b2, b3, bc)
            
            # Metal Gating (BCE Sigmoid * MSE Value)
            prob_re = torch.sigmoid(b_re_logits[0])
            val_re = m_re_vals[0]
            final_re = prob_re * val_re
            
            all_re_gen.append(sc1.inverse_transform(final_re.cpu().numpy()))
            all_re_true.append(sc1.inverse_transform(b1.cpu().numpy()))

    y_p_re = np.concatenate(all_re_gen).flatten()
    y_t_re = np.concatenate(all_re_true).flatten()
    
    current_r2 = r2_score(y_t_re, y_p_re)
    all_run_r2.append(current_r2)
    print(f"✅ Run {run:02d}/{NUM_RUNS} | Metal Reconstruction R2: {current_r2:.4f}")

# --- 3. 최종 결과 분석 및 시각화 ---

mean_r2 = np.mean(all_run_r2)
std_r2 = np.std(all_run_r2)

# 1. 결과 데이터 구조화 (JSON 저장용)
result_data = {
    "experiment_info": {
        "num_runs": NUM_RUNS,
        "epochs": EPOCHS,
        "device": str(device),
        "gamma_list": [0.01, 0.005, 0.002]
    },
    "metrics": {
        "all_run_r2": [float(r) for r in all_run_r2], # numpy float은 JSON 저장이 안 되므로 변환
        "average_r2": float(mean_r2),
        "std_r2": float(std_r2),
        "max_r2": float(np.max(all_run_r2)),
        "min_r2": float(np.min(all_run_r2))
    }
}

# 2. JSON 파일 저장
save_path = 'experiment_results.json'
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(result_data, f, indent=4, ensure_ascii=False)

print(f"\n📂 결과가 '{save_path}'에 저장되었습니다.")

# --- 기존 출력 및 시각화 로직 ---
print("\n" + "="*45)
print(f"📊 Final Report (over {NUM_RUNS} runs)")
print(f"Average Metal R2 : {mean_r2:.4f} (±{std_r2:.4f})")
print(f"Best R2: {np.max(all_run_r2):.4f} | Worst R2: {np.min(all_run_r2):.4f}")
print("="*45)

# Boxplot 시각화

plt.figure(figsize=(8, 6))
plt.boxplot(all_run_r2, patch_artist=True, boxprops=dict(facecolor='lightblue'))
plt.title(f'Metal Reconstruction $R^2$ Stability ({NUM_RUNS} Runs)')
plt.ylabel('$R^2$ Score')
plt.xticks([1], ['Metal Expert System'])
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

✅ 데이터 로드 성공!
🚀 Starting 20 Independent Runs on cpu...
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
E